# Traveling Salesman Problem - Genetski Algoritam

## Šta je TSP (Traveling Salesman Problem)?

Zadatak TSP-a je da jedan putujući trgovac prođe kroz svaki grad unutar regiona i da prođe najkraćim putem a da
ne svrati u isti grad dva puta.
Glavni problem TSP-a što ga čini veoma popularnim je pri većim količinama gradova kompleksnost TSP-a se značajno
povećava.

## Šta su genetski algoritmi?

Genetski algoritam predstavlja rešavanje jednog problema pomoću teorije evolucije.

### Inicijalizacija

Genetski algoritmi rade tako što se uzima određeni broj jedinki koji predstavljaju neko _validno_ rešenje koje
ne mora biti optimalno, pa se svakoj jedinki meri "fitness" pomoću heurističke funkcije. Heurističa funkcija
i "fitness" predstavljaju koliko je optimalno rešenje.

### Selekcija i rekombinacija

Zatim se od izmerenih "fitness" vrednosti generiše selekcija roditelja koji će napraviti sledeću populaciju.
kada se odrede roditelji geni im se ukrštaju i stvara se potomak sa novim genima i "fitness"-om.
Proces se završava kada se napravi nova generacija sa istim brojem populacije.

### Elitizam

Postoji šansa da je nova generacija gora od prošle, u tom slučaju možemo da primenimo elitizam i da osiguramo da barem
jedna najbolja jedinka iz prethodne generacije ostane.

### Mutacija
Pošto će populacija krenuti da se poboljšava u određenom pravcu postoji šansa da se dostigne lokalni maksimum što se tiče
"fitness"-a gde ni jedan pravac ne poboljšava jedinke iako postoji optimalnije rešenje. Zbog ovoga se uvodi mala šansa
mutacije među jedinki da ih izbaci iz mogućeg "ćoška".

![lokalni maksimum](./images/lokalni_maksimum.png)

Zatime se opet vrše svi ciklusi iznova onoliko generacija koliko mislimo da je dovoljno. Što više generacija, to duže će trebati da se izračuna ali se dobijaju bolji rezultati.

# Rešavanje TSP-a uz pomoć genetskog algoritma

## Julia

Algoritam je napisan u Julia jeziku. Jezik je mešavina raznih drugih programskih jezika kao što su Python, Matlab, R, i uzima neke koncepte od funkcionalnih jezika kao što su Haskell.

Julia se koristi najviše u matematičke i statističke svrhe pa je zato savršen jezik za rešavanje ovog problema.

## Predstavljanje podataka

Prvo je bitno kako đemo predstavljati podatke u rešenju. Najbitnije su sledeće strukture

In [31]:
struct Node
    x::Int
    y::Int
end

struct Individual
    genome::Vector{Node}
    fitness::Float64
    distance::Float64
end

`Node` predstavlja jednu koordinatu grada a `Individual` predstalja jedinku unutar populacije.
Unutar `Individual` strukture `genome` predstavlja moguće rešenje u vidu neponavljajućeg niza gradova
`fitness` predstavlja vrednost ili "jačinu" te jedinke (u ovom slučaju se računa kao 1/`distance`) a `distance`
predstavlja ukupno pređenu dužinu.

Zatim imamo neke korisne funkcije koje implementiramo da bih smo mogli koristiti neke specijalizovane funkcije

In [32]:
import Base

# Return the maximum of the two individuals
Base.max(g1::Individual, g2::Individual)::Individual = g1.fitness > g2.fitness ? g1 : g2

# Return true if the first individual is less than the second
Base.isless(g1::Individual, g2::Individual)::Bool = g1.fitness < g2.fitness

I usput dodajemo neke korisne funkcije za računanje razdaljine između dva grada i ukupnu dužinu putanje

In [33]:
# Returns the distance between nodes
distancebetween(n1::Node, n2::Node)::Float64 =
    sqrt(abs(n1.x - n2.x)^2 + abs(n1.y - n2.y)^2)

# Returns the total distance of a path
# SUM n1 -> n2 -> n3 -> ... -> nn
function distancetotal(genomes::Vector{Node})::Float64
    prev_node = genomes[1]
    distance = 0
    for i = 2:length(genomes)
        curr_node = genomes[i]
        distance += distancebetween(prev_node, curr_node)
        prev_node = curr_node
    end

    return distance
end

distancetotal (generic function with 1 method)

## Fitness

Kod računanja fitness-a uzimamo 1 / vrednost ukupne dužine putanje. Što je duža putanja to je manji fitness.

In [34]:
fitness(genomes::Vector{Node})::Float64 =
    1 / distancetotal(genomes)

fitness (generic function with 1 method)

Sada možemo napraviti funkciju koja generiše jedinke

In [35]:
using Random

# Generate a random valid solution by shuffling the nodes
# then create an individual
# Can you afford to be one though?
function makeindividual(nodes::Vector{Node})::Individual
    genomes = shuffle(nodes)
    fit = fitness(genomes)
    dist = distancetotal(genomes)

    return Individual(genomes, fit, dist)
end

makeindividual (generic function with 1 method)

In [36]:
cities = [Node(1, 1), Node(2, 2), Node(3, 3), Node(4, 4), Node(5, 5)];

i1 = makeindividual(cities)
i2 = makeindividual(cities)

println("i1: ", i1)
println("i2: ", i2)

println("Is i1 less than i2? ", isless(i1, i2))

i1: Individual(Node[Node(1, 1), Node(5, 5), Node(3, 3), Node(2, 2), Node(4, 4)], 0.07856742013183861, 12.727922061357857)
i2: Individual(Node[Node(4, 4), Node(2, 2), Node(5, 5), Node(1, 1), Node(3, 3)], 0.0642824346533225, 15.556349186104045)
Is i1 less than i2? false


## Mutacija

Da bi smo izvršili mutaciju jedne jedinke možemo jednostavno zameniti pozicije da grada u nizu i rezultat će još
biti validan

<img src="./images/mutacija.png" width="300" />

Što se tiče koda, prolazimo kroz sve gradove u nizu i testiramo da li je neki nasumičan broj manji od zadatog praga
mutacije, ako jeste uzimamo neku drugu nasumičnu poziciju i menjamo mesta tim gradovima.

> **Napomena**: ovim načinom postoji šansa da se zamene više od dva grada u jednoj mutaciji ali je veoma mala i izbalancira se
sa šansom da se zameni isti grad sa sobom tako da se ne desi mutacija.

In [37]:
function mutate(individual::Individual, mutation_rate::Float64)::Individual
    gen = copy(individual.genome)
    for i in 1:length(gen)
        if rand(Float64) < mutation_rate
            pos2 = rand(1:length(gen))
            
            tmp = gen[i]
            gen[i] = gen[pos2]
            gen[pos2] = tmp
        end
    end

    dist = distancetotal(gen)
    fit = fitness(gen)

    return Individual(gen, fit, dist)
end

mutate (generic function with 1 method)

In [38]:
println("Before mutation: ", i1)
println("After mutation: ", mutate(i1, 0.05))

Before mutation: Individual(Node[Node(1, 1), Node(5, 5), Node(3, 3), Node(2, 2), Node(4, 4)], 0.07856742013183861, 12.727922061357857)
After mutation: Individual(Node[Node(1, 1), Node(5, 5), Node(3, 3), Node(2, 2), Node(4, 4)], 0.07856742013183861, 12.727922061357857)


## Rekombinacija

Kod rekombinacije postupak je malo komplikovaniji i predstavlja jedan "teži" deo koda.

Prvi deo je da uzmemo jedan "isečak" putanje iz prvog roditelja i postavljamo ga u istu poziciju kod deteta:

<img src="./images/rekombinacija1.png" width=300 />

Drugi korak je gde prolazimo kroz sve gradove redom iz drugog roditelja i kopiramo one koji već ne postoje nad toj poziciji
u dete, vodeći računa u kom redosledu ih postavljamo:

<img src="./images/rekombinacija2.png" width=300 />

Kod radi na sličnom principu tako što uzima jedan opseg kod prvog roditelja, zatim prolazi kroz drugog roditelja i
postavlja gradove u slobodne pozicije kod deteta onim redosledom kojim se pojavljuju ako ne postoje već u
prvom roditelju, tako osiguravamo da je rezultat još uvek validan

In [39]:
function crossover(individual1::Individual, individual2::Individual)::Individual
    # find a "window" to take from the first parent
    (pos1, pos2) = minmax(rand(1:length(individual1.genome)), rand(1:length(individual1.genome)))

    # Save the segment we want to transfer to the child
    seqm = individual1.genome[pos1:pos2]
    
    gen = Array{Node}(undef, length(individual2.genome)) 

    j = 1
    # Go through every node in the second parent and add it to the
    # child in order of appearence if it doesn't exist in the first
    # parents selected genes
    for i = 1:length(individual2.genome)
        if !(individual2.genome[i] in seqm)
            # goes through the child until it finds a free position
            while j <= length(gen)
                if (j in pos1:pos2)
                    j += 1
                else
                    gen[j] = individual2.genome[i]
                    j += 1
                    break
                end
            end
        end
    end

    # actually apply the first parents genes and create the child
    gen[pos1:pos2] = seqm
    fit = fitness(gen)
    dist = distancetotal(gen)

    return Individual(gen, fit, dist)
end

crossover (generic function with 1 method)

In [40]:
println("Parent 1: ", i1)
println("Parent 2: ", i2)
println("Crossover: ", crossover(i1, i2))

Parent 1: Individual(Node[Node(1, 1), Node(5, 5), Node(3, 3), Node(2, 2), Node(4, 4)], 0.07856742013183861, 12.727922061357857)
Parent 2: Individual(Node[Node(4, 4), Node(2, 2), Node(5, 5), Node(1, 1), Node(3, 3)], 0.0642824346533225, 15.556349186104045)
Crossover: Individual(Node[Node(4, 4), Node(5, 5), Node(3, 3), Node(2, 2), Node(1, 1)], 0.1414213562373095, 7.0710678118654755)


## Selekcija

Jedan od najvažnijih koraka je selekcija jedinki za pravljenje nove populacije. Sistem selekcije predstavlja jednu _lutriju_
gde svakoj jedinki dodeljuje jedan deo na "pikado" tabli, veličina dela zavisi od jačine te jedinke u odnosu na ukupnu
jačinu populacije: Npr ako imamo 4 jedinki:

| Jedinka | Fitness | Procenat kontribucije |
|:--------|---------|----------------------:|
| 1       | 0.5     | 50%                   |
| 2       | 0.25    | 25%                   |
| 3       | 0.125   | 12.5%                 |
| 4       | 0.125   | 12.5%                 |

Možemo napraviti sledeću "pikado" tabelu:

<img src="./images/pikado.png" width=300 />

Posle generisanja tabele možemo uzeti jedno roditelja tako što nasumično bacamo "strelu" u tabelu i uzimamo jedinku koju
pogodimo.

Ovu lutriju u kodu predstavljamo tako što dodeljujemo svakoj jedinki opseg pozicije od 1 do 1000 (npr. jedinka 1 ima opseg
1:500, jedinka dva ima opseg 501:750, itd.) zatim generišemo nasumičan broj od 1 do 1000 i gledamo koja jedinka ima taj
opseg:

In [41]:
function selectparent(population::Vector{Individual})::Individual
    # sum all of the fitnesses together
    totalfitness = reduce((x, y) -> x + y.fitness, population; init=0.0)
    lottery = []
    maxpoints = 1000
    minpoints = 1

    # genrate a range for every individual for the lottery
    for ind in population
        chance = ind.fitness / totalfitness
        points = Int(round(maxpoints * chance)) + minpoints
        push!(lottery, minpoints:points)

        minpoints = points
    end

    # the winning number
    randselection = rand(1:maxpoints)

    # find the selected parent
    for (i, value) in enumerate(lottery)
        if randselection in value
            return population[i]
        end
    end

    return population[rand(1:length(population))]
end

selectparent (generic function with 1 method)

In [42]:
population = collect((makeindividual(cities) for _ in 1:8))

display(population)

parent1 = selectparent(population)
parent2 = selectparent(population)

println("Parent 1: ", parent1)
println("Parent 2: ", parent2)

8-element Array{Individual,1}:
 Individual(Node[Node(4, 4), Node(3, 3), Node(5, 5), Node(1, 1), Node(2, 2)], 0.08838834764831842, 11.313708498984763)
 Individual(Node[Node(1, 1), Node(4, 4), Node(3, 3), Node(5, 5), Node(2, 2)], 0.07856742013183861, 12.727922061357855)
 Individual(Node[Node(4, 4), Node(3, 3), Node(1, 1), Node(2, 2), Node(5, 5)], 0.10101525445522107, 9.899494936611665)
 Individual(Node[Node(5, 5), Node(4, 4), Node(2, 2), Node(1, 1), Node(3, 3)], 0.1178511301977579, 8.485281374238571)
 Individual(Node[Node(2, 2), Node(4, 4), Node(5, 5), Node(1, 1), Node(3, 3)], 0.07856742013183861, 12.727922061357857)
 Individual(Node[Node(5, 5), Node(4, 4), Node(3, 3), Node(1, 1), Node(2, 2)], 0.1414213562373095, 7.0710678118654755)
 Individual(Node[Node(4, 4), Node(1, 1), Node(2, 2), Node(5, 5), Node(3, 3)], 0.07856742013183862, 12.727922061357853)
 Individual(Node[Node(3, 3), Node(4, 4), Node(2, 2), Node(1, 1), Node(5, 5)], 0.08838834764831843, 11.313708498984761)

Parent 1: Individual(Node[Node(4, 4), Node(1, 1), Node(2, 2), Node(5, 5), Node(3, 3)], 0.07856742013183862, 12.727922061357853)
Parent 2: Individual(Node[Node(5, 5), Node(4, 4), Node(3, 3), Node(1, 1), Node(2, 2)], 0.1414213562373095, 7.0710678118654755)


## Sve zajedno

Sledeće je da se svi ovi koraci redosledno primene: `selekcija -> rekombinacija -> mutacija`

In [43]:
function runcycle(population::Vector{Individual}; elitism=false, mutationrate=0.05)::Vector{Individual}
    popcount = length(population)
    newpop = []

    if elitism
        push!(newpop, maximum(population))
        popcount -= 1
    end

    while popcount != 0
        p1 = selectparent(population)
        p2 = selectparent(population)

        while p1 == p2
            p2 = selectparent(population)
        end

        child = crossover(p1, p2)
        push!(newpop, child)
        popcount -= 1
    end

    map(x -> mutate(x, mutationrate), newpop)

    return newpop
end

runcycle (generic function with 1 method)

> ### Elitizam
> Postoji šansa da jedinke nove generacije bude gore od najgore jedinke iz prethodne, tako da imamo i opciju za elitizam.
> Kada je elitizam aktivan čuva se najjača jedinka iz prethodne generacije.

Sada možemo da narpavimo jednostavnu funkciju koja će simulirati određeni broj generacija i vratiti najoptimalnije rešenje

In [44]:
function evolve(
    nodes::Vector{Node};
    indcount=8,
    iterations=1000,
    elitism=false,
    mutationrate=0.01)::Individual
    
    if indcount < 2
        throw(DomainError(indcount, "the number of individuals must be 2 or more"))
    end

    initpop::Vector{Individual} = []
    for _ = 1:indcount
        push!(initpop, makeindividual(nodes))
    end

    pop = initpop

    for _ = 1:iterations
        pop = runcycle(pop; elitism, mutationrate)
    end

    return maximum(pop)
end

evolve (generic function with 1 method)

# Krajnji rezultat

Sada možemo da nađemo potecijalno najkraći put kroz koji trgovac mora da prođe. Testiraćemo sa manjim brojem ciklusa, većim,
elitizmom, bez elitizma.

Početni podaci:

In [45]:
cities = [
    Node(1, 1),
    Node(1, 10),
    Node(5, 6),
    Node(8, 11),
    Node(4, 3),
    Node(2, 9),
    Node(5, 12)
];

## Bez elitizma, malo iteracija

In [46]:
evolve(
    cities;
    indcount = 8,
    iterations = 600,
    elitism = false
)

Individual(Node[Node(5, 12), Node(8, 11), Node(1, 1), Node(4, 3), Node(5, 6), Node(1, 10), Node(2, 9)], 0.03423751175455418, 29.207730023399925)

## Sa elitizmom, malo iteracija

In [47]:
evolve(
    cities;
    indcount = 8,
    iterations = 600,
    elitism = true
)

Individual(Node[Node(5, 12), Node(1, 10), Node(2, 9), Node(8, 11), Node(5, 6), Node(4, 3), Node(1, 1)], 0.04030683876347041, 24.8096856681871)

## Mnogo iteracija, bez elitizma

In [48]:
evolve(
    cities;
    indcount = 8,
    iterations = 10000,
    elitism = false
)

Individual(Node[Node(1, 1), Node(4, 3), Node(2, 9), Node(1, 10), Node(5, 12), Node(8, 11), Node(5, 6)], 0.04030683876347041, 24.8096856681871)

## Mnogo iteracija, sa elitizmom

In [53]:
evolve(
    cities;
    indcount = 8,
    iterations = 10000,
    elitism = true
)

Individual(Node[Node(1, 1), Node(4, 3), Node(5, 6), Node(1, 10), Node(2, 9), Node(5, 12), Node(8, 11)], 0.047072524192957196, 21.243815094785507)

## Rezultati

Rezultati će naravno varirati od izvršavanja do izvršavanja pa će nekada biti bliže optimalnom rešenju nekada dalje jer
algoritam zavisi od nasumičnosti. Ali možemo videti sa jednostavnijim primerom da najverovatnije će naći optimalno
rešenje:

In [54]:
cities_easy = [
    Node(1, 0),
    Node(2, 0),
    Node(5, 0),
    Node(4, 0),
    Node(3, 0)
];

evolve(
    cities_easy,
    indcount = 8,
    iterations = 10000,
    elitism = true
)

Individual(Node[Node(1, 0), Node(2, 0), Node(3, 0), Node(4, 0), Node(5, 0)], 0.25, 4.0)

Kao što vidimo rezultat će biti 1 -> 2 -> 3 -> 4 -> 5 (ili obrnuto), sem ako smo neopisivo nerećni (ili srećni u zavisnosti
kako gledamo), tako da znamo da algoritam radi.